In [1]:
import sys
import os
import torch
import cv2 as cv
import numpy as np
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import random
from torch.utils.data.sampler import SubsetRandomSampler
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import torchvision.models as models


from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Using GPU
print(sys.version)
device = 'cuda'
#Checking for GPU
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.is_available())

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
10.1
0
False


In [4]:
lookup = dict()
reverselookup = dict()

count = 0

for j in sorted(os.listdir('drive/My Drive/Colab Notebooks/ITSP/dynamicdata/')):
    if not j.startswith('.'):
        lookup[j] = count
        reverselookup[count] = j
        count += 1
        
print(lookup)
print(reverselookup)

{'close': 0, 'fingerright': 1, 'fistround': 2, 'palmdown': 3, 'palmleft': 4, 'scrollup': 5}
{0: 'close', 1: 'fingerright', 2: 'fistround', 3: 'palmdown', 4: 'palmleft', 5: 'scrollup'}


In [18]:
num_classes = len(reverselookup)
video_num = 100
frame_count = 16

path = 'drive/My Drive/Colab Notebooks/ITSP/dynamicdata/'

image_size = 224

transform = transforms.Compose([
                transforms.Resize(image_size),
                transforms.RandomResizedCrop(size=image_size, scale=(0.95, 1.0)),
                transforms.ColorJitter(),
                transforms.ToTensor(), 
                transforms.Normalize([0.5], [0.5])
                ])
videoset = []
videoclass = []

for i in range(num_classes):
  for k in range(video_num):
    gest = None
    for j in range(frame_count):
      #print(path+str(reverselookup[i])+'/'+str(k+1)+'/'+str(j+1)+'.jpg')
      img = Image.open(path+str(reverselookup[i])+'/'+str(k+1)+'/'+str(j+1)+'.jpg').convert('RGB')
      
      #img = cv.cvtColor(img.numpy(), cv.COLOR_GRAY2RGB)
      img = transform(img)
      img = img.view(3, image_size, image_size, 1)
      #print(img.shape)
      
      if gest is None:
        gest = img
      else:
        gest = torch.cat((gest, img), dim=3)
    print('.', end=' ')
    videoset.append(gest)
  videoclass.append(videoset)
  print(len(videoset))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 100
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 200
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 300
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 400
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [19]:
class GestureDataset(Dataset):
    """Dynamic Gesture dataset."""

    def __init__(self, tensor_list, num_classes=6, num_videos=100):

        self.tensor_list = tensor_list
        self.num_classes = num_classes
        self.num_videos = num_videos
        #self.transform = transform

    def __len__(self):
        return len(self.tensor_list)

    def __getitem__(self, idx):
        vid = self.tensor_list[idx]
        label = int(idx/self.num_videos)

        return (vid, label)

dataset = GestureDataset(videoset)
print(dataset)
torch.save(dataset, 'drive/My Drive/Colab Notebooks/ITSP/ds_dynamicGestrnn2.pth')

In [20]:
'''num_classes = len(reverselookup)
video_num = 100
frame_count = 16

path = 'drive/My Drive/Colab Notebooks/ITSP/dynamicdata/'

train_loader = torch.load('drive/My Drive/Colab Notebooks/ITSP/dl_train_dynamicGestrnn.pth')
valid_loader = torch.load('drive/My Drive/Colab Notebooks/ITSP/dl_valid_dynamicGestrnn.pth')'''

"num_classes = len(reverselookup)\nvideo_num = 100\nframe_count = 16\n\npath = 'drive/My Drive/Colab Notebooks/ITSP/dynamicdata/'\n\ntrain_loader = torch.load('drive/My Drive/Colab Notebooks/ITSP/dl_train_dynamicGestrnn.pth')\nvalid_loader = torch.load('drive/My Drive/Colab Notebooks/ITSP/dl_valid_dynamicGestrnn.pth')"

In [21]:
'''random_seed = random.randint(1, 100)
batch = 8

print(len(train_loader))
print(len(valid_loader))'''

'random_seed = random.randint(1, 100)\nbatch = 8\n\nprint(len(train_loader))\nprint(len(valid_loader))'

In [22]:
'''dataset = torch.load('drive/My Drive/Colab Notebooks/ITSP/ds_dynamicGestrnn.pth')'''

"dataset = torch.load('drive/My Drive/Colab Notebooks/ITSP/ds_dynamicGestrnn.pth')"

In [23]:
random_seed = random.randint(1, 100)
batch = 8
num_classes = len(lookup)

dataset_size = len(dataset)
split = int(0.05*dataset_size)
print('Size of dataset is: ', dataset_size)
print('Split value: ', split)
indices = list(range(dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch, sampler=train_sampler,
                                            num_workers = 2)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch, sampler=valid_sampler,
                                            num_workers = 2)

print(len(train_loader))
print(len(valid_loader))

Size of dataset is:  600
Split value:  30
72
4


In [27]:
torch.save(train_loader, 'drive/My Drive/Colab Notebooks/ITSP/dl_train_dynamicGestrnn2.pth')
torch.save(valid_loader, 'drive/My Drive/Colab Notebooks/ITSP/dl_valid_dynamicGestrnn2.pth')

In [ ]:
torch.save(train_loader, 'drive/My Drive/Colab Notebooks/ITSP/dl_train_dynamicGestrnn5.pth')
torch.save(valid_loader, 'drive/My Drive/Colab Notebooks/ITSP/dl_valid_dynamicGestrnn5.pth')

In [26]:
# Iterate through the dataloader once
trainiter = iter(train_loader)
features, labels = next(trainiter)
#features.shape, labels.shape
print(features.shape)

for f, l in train_loader:
  print(l)

for f, l in valid_loader:
  print(l)

torch.Size([8, 3, 224, 224, 16])
tensor([2, 2, 2, 2, 0, 5, 4, 0])
tensor([0, 5, 5, 2, 1, 2, 0, 2])
tensor([0, 2, 1, 2, 3, 3, 3, 4])
tensor([2, 2, 5, 1, 1, 3, 0, 4])
tensor([3, 3, 1, 1, 2, 0, 1, 1])
tensor([4, 5, 4, 5, 4, 2, 2, 3])
tensor([5, 4, 1, 3, 0, 3, 5, 4])
tensor([2, 0, 4, 0, 4, 4, 3, 0])
tensor([4, 2, 5, 0, 1, 3, 4, 4])
tensor([1, 1, 0, 5, 3, 2, 0, 0])
tensor([5, 1, 3, 5, 3, 4, 4, 2])
tensor([3, 0, 5, 1, 1, 3, 3, 5])
tensor([0, 1, 5, 1, 1, 5, 0, 5])
tensor([1, 1, 5, 0, 4, 5, 2, 4])
tensor([4, 2, 3, 1, 3, 4, 2, 1])
tensor([3, 5, 2, 3, 3, 1, 3, 3])
tensor([2, 5, 4, 5, 2, 5, 4, 0])
tensor([2, 5, 5, 2, 0, 2, 4, 2])
tensor([4, 2, 3, 2, 1, 3, 0, 0])
tensor([2, 1, 1, 4, 5, 5, 0, 3])
tensor([5, 3, 0, 2, 5, 2, 3, 3])
tensor([4, 4, 3, 1, 5, 3, 5, 2])
tensor([1, 5, 0, 2, 4, 0, 0, 2])
tensor([2, 2, 1, 0, 4, 1, 4, 2])
tensor([4, 5, 3, 3, 1, 4, 1, 3])
tensor([0, 1, 3, 0, 0, 0, 4, 4])
tensor([5, 2, 4, 1, 3, 3, 5, 3])
tensor([4, 5, 1, 1, 1, 0, 2, 3])
tensor([3, 1, 0, 3, 1, 2, 0, 3])
tensor([3,

In [ ]:
'''model1 = models.resnet101(pretrained=True)
for param in model1.parameters():
    param.requires_grad = False
print(model1)'''

In [ ]:
# EncoderCNN architecture
CNN_fc_hidden1, CNN_fc_hidden2 = 1024, 768
CNN_embed_dim = 512      # latent dim extracted by 2D CNN
img_x, img_y = 256, 342  # resize video 2d frame size
dropout_p = 0.0          # dropout probability

# DecoderRNN architecture
RNN_hidden_layers = 3
RNN_hidden_nodes = 512
RNN_FC_dim = 256

In [ ]:
# 2D CNN encoder using ResNet-101 pretrained
class ResCNNEncoder(nn.Module):
    def __init__(self, fc_hidden1=512, fc_hidden2=512, drop_p=0.3, CNN_embed_dim=300):
        """Load the pretrained ResNet-101 and replace top fc layer."""
        super(ResCNNEncoder, self).__init__()

        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p

        resnet = models.resnet101(pretrained=True)                 

        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.fc1 = nn.Linear(resnet.fc.in_features, fc_hidden1)
        #self.bn1 = nn.BatchNorm1d(fc_hidden1, momentum=0.01)
        self.fc2 = nn.Linear(fc_hidden1, fc_hidden2)
        #self.bn2 = nn.BatchNorm1d(fc_hidden2, momentum=0.01)
        self.fc3 = nn.Linear(fc_hidden2, CNN_embed_dim)
        
    def forward(self, x_3d):
        cnn_embed_seq = []
        x = x_3d.view(-1, 16, 3, 224, 224)
        for t in range(x_3d.size(1)):
            # ResNet CNN
            with torch.no_grad():
                x = self.resnet(x_3d[:, t, :, :, :])  # ResNet
                x = x.view(x.size(0), -1)             # flatten output of conv

            # FC layers
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)

            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq


class DecoderRNN(nn.Module):
    def __init__(self, CNN_embed_dim=300, h_RNN_layers=3, h_RNN=256, h_FC_dim=128, drop_p=0.3, num_classes=6):
        super(DecoderRNN, self).__init__()

        self.RNN_input_size = CNN_embed_dim
        self.h_RNN_layers = h_RNN_layers   # RNN hidden layers
        self.h_RNN = h_RNN                 # RNN hidden nodes
        self.h_FC_dim = h_FC_dim
        self.drop_p = drop_p
        self.num_classes = num_classes

        self.LSTM = nn.LSTM(
            input_size=self.RNN_input_size,
            hidden_size=self.h_RNN,        
            num_layers=h_RNN_layers,       
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.fc1 = nn.Linear(self.h_RNN, self.h_FC_dim)
        self.fc2 = nn.Linear(self.h_FC_dim, self.num_classes)

    def forward(self, x_RNN):
        
        self.LSTM.flatten_parameters()
        RNN_out, (h_n, h_c) = self.LSTM(x_RNN, None)  
        """ h_n shape (n_layers, batch, hidden_size), h_c shape (n_layers, batch, hidden_size) """ 
        """ None represents zero initial hidden state. RNN_out has shape=(batch, time_step, output_size) """

        # FC layers
        x = self.fc1(RNN_out[:, -1, :])   # choose RNN_out at the last time step
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc2(x)

        return x

## ---------------------- end of CRNN module ---------------------- ##

In [ ]:
cnn_encoder = ResCNNEncoder()

rnn_decoder = DecoderRNN()

In [ ]:
total_params = sum(p.numel() for p in cnn_encoder.parameters())

print(f'{total_params:,} total parameters in CNN')

total_trainable = sum(p.numel() for p in cnn_encoder.parameters() if p.requires_grad)

print(f'{total_trainable:,} trainable parameters in CNN.')


total_params = sum(p.numel() for p in rnn_decoder.parameters())

print(f'{total_params:,} total parameters in RNN')

total_trainable = sum(p.numel() for p in rnn_decoder.parameters() if p.requires_grad)

print(f'{total_trainable:,} trainable parameters in RNN.')

In [26]:
# Move to gpu
cnn_encoder = cnn_encoder.to('cuda')
rnn_decoder = rnn_decoder.to('cuda')

loss_func = F.cross_entropy

learning_rate = 0.0001
#optimizer = optim.Adam(model.parameters(), betas=(0.5, 0.99), lr = 0.0001)
crnn_params = list(cnn_encoder.parameters()) + list(rnn_decoder.parameters())
optimizer = torch.optim.Adam(crnn_params, lr=learning_rate)

In [27]:
def accuracy(out, yb):
    #print(yb)
    pred = torch.argmax(out, keepdim= False, dim=1)
    #print(out.shape, pred)
    return (pred == yb).float().mean() 

In [28]:
def fit(model, epochs, train_loader, valid_loader, opt):
    losses = []
    
    for epoch in range(epochs):
        train_loss =0
        train_acc =0
        cnn_encoder, rnn_decoder = model
        cnn_encoder.train()
        rnn_decoder.train()
        for xb, yb in train_loader:
            xb =xb.to(device)
            yb = yb.to(device)
            pred = rnn_decoder(cnn_encoder(xb.float()))
            pred.to(device)
            train_acc += accuracy(pred, yb)
            loss = loss_func(pred, yb)
            train_loss += loss
            #back propogation
            loss.backward()
            opt.step()
            opt.zero_grad()      

        losses.append(train_loss)
        print("Iteration no: "+ str(epoch), "loss = "+str(losses[epoch].item()))
        print("Accuracy of train set:", train_acc/len(train_loader))
        lr_scheduler.step(train_loss/len(train_loader))
        
        
        cnn_encoder, rnn_decoder
        cnn_encoder.eval()
        rnn_decoder.eval()    
        with torch.no_grad():
            valid_acc=0
            loss_valid = 0
            for xb_valid, yb_valid in valid_loader:
                xb_valid = xb_valid.to(device)
                yb_valid = yb_valid.to(device)
                pred_valid = rnn_decoder(cnn_encoder(xb_valid.float()))
                pred_valid.to(device)
                valid_acc += accuracy(pred_valid, yb_valid)
                loss_valid += loss_func(pred_valid, yb_valid)
            print("Accuracy of validation set :", valid_acc/len(valid_loader))
            #callbacks = [EarlyStopping(monitor='valid_acc', verbose=False, mode='max', patience=10)]

    plt.plot(losses)
    plt.ylabel("loss")
    plt.xlabel("iterations")
    plt.show()

In [29]:
#model.set_callbacks(callbacks)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True, eps = 10e-15)

In [30]:
model = (cnn_encoder, rnn_decoder)
fit(model, 35, train_loader, valid_loader, optimizer)

RuntimeError: ignored

In [ ]:
torch.save(model, 'drive/My Drive/Colab Notebooks/ITSP/c3d6.pth')